In [2]:
import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

# 종목코드가 6자리이기 때문에 6자리를 위한 설정
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

# 필요없는 column 제외해서 출력
code_df = code_df[['회사명', '종목코드']]

#한글로된 컬럼명을 영어로 바꿈
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
code_df

,name,code
0,DSR,155660
1,GS글로벌,001250
2,HDC현대산업개발,294870
3,KG케미칼,001390
4,LG이노텍,011070
...,...,...
2401,툴젠,199800
2402,플럼라인생명과학,222670
2403,피엔에이치테크,239890
2404,한국미라클피플사,331660


In [67]:
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code.strip()) 
    
    print("요청 URL = {}".format(url)) 
    return url

# 주식가격의 일자데이터 url 가져오기
item_name = 'CJ프레시웨이'
url = get_url(item_name, code_df)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=051500


In [68]:
# 일자 데이터를 담을 df라는 DataFrame 정의
df = pd.DataFrame()

# 원하는 페이지 불러오기
for page in range(1, 173):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna()

# 상위 5개데이터 확인
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.12.21,19100.0,150.0,18800.0,19100.0,18700.0,87517.0
2,2020.12.18,18950.0,150.0,19150.0,19150.0,18900.0,46893.0
3,2020.12.17,19100.0,50.0,19100.0,19150.0,18750.0,65737.0
4,2020.12.16,19050.0,200.0,19000.0,19150.0,18800.0,71174.0
5,2020.12.15,18850.0,250.0,19100.0,19150.0,18700.0,93468.0


In [69]:
# 한글로된 컬럼명을 영어로
df = df.rename(columns={'날짜':'date', '종가':'close', '전일비': 'diff',
                       '시가':'open', '고가':'high', '저가':'low', '거래량':'volume'})
# 데이터타입 변경: int
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
= df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)

# 컬럼명 'date'의 타입을 date로 변경
df['date'] = pd.to_datetime(df['date'])

# 일자(date)를 기준으로 오름차순 정렬
df = df.sort_values(by=['date'], ascending=True)

df.head(10)

,date,close,diff,open,high,low,volume
2578,2013-12-20,29200,1150,27950,30400,27600,18226
2577,2013-12-23,30250,1050,29200,30250,29200,33442
2576,2013-12-24,30050,200,30300,30500,29950,14620
2575,2013-12-26,29500,550,29850,30400,29400,7912
2574,2013-12-27,29300,200,29500,29550,29000,9052
2570,2013-12-30,29300,0,29950,29950,29300,7315
2569,2014-01-02,29250,50,29450,29500,28800,5519
2568,2014-01-03,29550,300,29250,29550,28550,9319
2567,2014-01-06,29700,150,29950,29950,28650,7884
2566,2014-01-07,28700,1000,29400,29700,28700,10576


In [70]:
df.to_csv("cjfresh_stock.csv")